In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,roc_curve,confusion_matrix,roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
import joblib as jd

In [14]:
df=pd.read_csv("C:\\Users\\user\\Downloads\\bank_recommendation_data.csv")
df.sample(5)

,customer_id,age,monthly_income,employment_years,credit_score,existing_loans,monthly_expenses,account_balance,fraud_risk_score,recommended_product
0,1,59,1767,11,600,0,1700,16896,0.42,Credit Limit Increase
22,23,32,1428,1,657,1,926,13296,0.05,Loan
39,40,29,1459,1,833,0,1192,4611,0.97,Salary Advance
31,32,27,1263,7,553,0,1565,2849,0.09,Savings Account
66,67,41,1083,6,679,1,940,417,0.38,Salary Advance


In [15]:
encoder= LabelEncoder()
df['recommended_product']=encoder.fit_transform(df['recommended_product'])
mapping=dict(zip(encoder.classes_,encoder.transform(encoder.classes_)))
mapping

{'Credit Limit Increase': np.int64(0),
 'Loan': np.int64(1),
 'Salary Advance': np.int64(2),
 'Savings Account': np.int64(3)}

In [18]:
x=df[['credit_score','existing_loans','fraud_risk_score','monthly_expenses']]
y=df['recommended_product']

In [20]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)
forest=RandomForestClassifier(n_estimators=40,random_state=42,class_weight='balanced')

In [21]:
forest.fit(x_train,y_train)

,n_estimators,40
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [29]:
y_pred=forest.predict(x_test)
y_pred

array([1, 1, 0, 0, 3, 1, 0, 0, 1, 1, 0, 0, 0, 0, 3])

In [35]:
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='macro')  
recall = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)


Accuracy: 0.4
Precision: 0.3
Recall: 0.35416666666666663
F1-score: 0.3234126984126984


C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


In [38]:
joblib.dump(encoder,'encoder.pkl')

['encoder.pkl']

In [41]:
from fastapi import FastAPI
from pydantic import BaseModel
model = joblib.load('recommended_product.pkl')
encoder=joblib.load('encoder.pkl')
app = FastAPI()

class Customer(BaseModel):
    credit_score: float
    existing_loans: int
    fraud_risk_score: float
    monthly_expenses: float

@app.post("/recommend/")
def recommend(customer: Customer):
    data = pd.DataFrame([customer.dict()])
    pred = model.predict(data)
    product_name = encoder.inverse_transform(pred)[0]
    return {"recommended_product": product_name}
